In [2]:
import numpy as np
import pandas as pd

### Pdf reading

In [3]:
%pip install --upgrade pdfplumber pdfminer -q

import pdfplumber

Note: you may need to restart the kernel to use updated packages.


In [4]:
def _convert_pdf_to_text(path):
    """Convert the PDF to text and store it in the self.content
    attribute.
    """
    text = []
    pdf = pdfplumber.open(path)
    for page in range(len(pdf.pages)):
        # get the page object
        page_obj = pdf.pages[page]
        # extract text from the page
        text.append(
            page_obj.extract_text(
                x_tolerance=1, x_tolerance_ratio=None, y_tolerance=1, layout=False, x_density=7.25, y_density=13, line_dir_render=None,keep_blank_chars=True,use_text_flow=True, char_dir_render=None, expand_ligatures=True
            )
        )
    text = "\n".join(text)
    
    return text

In [5]:
path = "D:\Projects\LLM\Createdb\docs\\aco and pso.pdf"
text = _convert_pdf_to_text(path)

In [6]:
print(text)

Journal of Constructional Steel Research 65 (2009) 1558–1568
Contents lists available at ScienceDirect
Journal of Constructional Steel Research
journal homepage: www.elsevier.com/locate/jcsr
A particle swarm ant colony optimization for truss structures with
discrete variables
A. Kaveh
a,∗
,1
, S. Talatahari
b
a
Institute for Mechanics of Materials and Structures, Vienna University of Technology, Karlsplatz 13, A-1040 Wien, Austria
b
Department of Civil Engineering, Tabriz University, Tabriz, Iran
a r t i c l e i n f o
Article history:
Received 19 March 2009
Accepted 24 April 2009
Keywords:
Heuristic particle swarm ant colony
optimization
Harmony search
Optimum truss design
Discrete variables
a b s t r a c t
In this paper, a particle swarm optimizer with passive congregation (PSOPC), ant colony optimization
(ACO) and harmony search scheme (HS) are combined to reach to an efficient algorithm, called discrete
heuristic particle swarm ant colony optimization (DHPSACO). This method is then 

In [7]:
text = text.lower()

In [8]:
print(text)

journal of constructional steel research 65 (2009) 1558–1568
contents lists available at sciencedirect
journal of constructional steel research
journal homepage: www.elsevier.com/locate/jcsr
a particle swarm ant colony optimization for truss structures with
discrete variables
a. kaveh
a,∗
,1
, s. talatahari
b
a
institute for mechanics of materials and structures, vienna university of technology, karlsplatz 13, a-1040 wien, austria
b
department of civil engineering, tabriz university, tabriz, iran
a r t i c l e i n f o
article history:
received 19 march 2009
accepted 24 april 2009
keywords:
heuristic particle swarm ant colony
optimization
harmony search
optimum truss design
discrete variables
a b s t r a c t
in this paper, a particle swarm optimizer with passive congregation (psopc), ant colony optimization
(aco) and harmony search scheme (hs) are combined to reach to an efficient algorithm, called discrete
heuristic particle swarm ant colony optimization (dhpsaco). this method is then 

### replacing topics

In [9]:
import re

def pa(text):
    # text = "abstract  abstract  abstract"
    pattern_abstract = r'\s*a\s*b\s*s\s*t\s*r\s*a\s*c\s*t\s*'
    matches1 = re.findall(pattern_abstract,text)
    print(matches1)
    match = ''.join(matches1[0])
    print(match)
    pattern_abstract = re.compile(pattern_abstract)
    return pattern_abstract.sub('\nabstract\n',text)
    # matches1 = re.findall(pattern_abstract, text)
    # text.replace(match,'abstract')
def pi(text):
    # text = "abstract  abstract  abstract"
    pattern_intro = r'i\s*n\s*t\s*r\s*o\s*d\s*u\s*c\s*t\s*i\s*o\s*n'
    matches1 = re.findall(pattern_intro,text)
    print(matches1)
    match = ''.join(matches1[0])
    print(match)
    pattern_intro = re.compile(pattern_intro)
    return pattern_intro.sub('\nintroduction\n',text)
    # matches1 = re.findall(pattern_abstract, text)
    # text.replace(match,'abstract')
def pr(text):
    # text = "abstract  abstract  abstract"
    pattern_intro = r'r\s*e\s*f\s*e\s*r\s*e\s*n\s*c\s*e\s*s'
    matches1 = re.findall(pattern_intro,text)
    if len(matches1) == 0:
        return text 
    print(matches1)
    match = ''.join(matches1[0])
    print(match)
    pattern_intro = re.compile(pattern_intro)
    return pattern_intro.sub('\nreferences\n',text)
    # matches1 = re.findall(pattern_abstract, text)
    # text.replace(match,'abstract')
text = pa(text)
text = pi(text)
text = pr(text)

['\na b s t r a c t\n']

a b s t r a c t

['introduction']
introduction
['references']
references


In [10]:
data = [[text]]

In [11]:
df = pd.DataFrame(data, columns=['raw_text'])

### adding title from first page

In [12]:
# may not be accurate 
# we are assuming that the text with the largest font is the title


import pdfplumber

def find_text_by_largest_font(page):
    largest_font_size = 0
    largest_font_chars = []

    for char in page.chars:
        size = char["size"]
        if size > largest_font_size:
            largest_font_size = size
            largest_font_chars = [char]
        elif size == largest_font_size:
            largest_font_chars.append(char)
    # for char in page.chars:
    #     size = char["size"]
    #     if size == largest_font_size or char == " ":
    #         largest_font_chars.append(char)

    return largest_font_chars


with pdfplumber.open(path) as pdf:
    first_page = pdf.pages[0]
    largest_font_chars = find_text_by_largest_font(first_page)

    largest_font_text = ''.join(char['text'] for char in largest_font_chars)

    # print("-----")
    # print(largest_font_text)
    # print("-----")
df['Title'] = largest_font_text

In [13]:
df

,raw_text,Title
0,journal of constructional steel research 65 (2...,JournalofConstructionalSteelResearch


### String Slicing for Title

In [14]:
from nltk.tokenize import word_tokenize

new = word_tokenize(text)

In [15]:
# may or may not work depending on the dictionary



import numpy as np
from wordfreq import get_frequency_dict

word_prob = get_frequency_dict(lang='en', wordlist='large')
max_word_len = max(map(len, word_prob))  # 34

# def str_slice(text):
def viterbi_segment(text, debug=False):
    probs, lasts = [1.0], [0]
    for i in range(1, len(text) + 1):
        new_probs = []
        for j in range(max(0, i - max_word_len), i):
            substring = text[j:i]
            length_reward = np.exp(len(substring))
            freq = word_prob.get(substring, 0) * length_reward
            compounded_prob = probs[j] * freq
            new_probs.append((compounded_prob, j))
            
            if debug:
                print(f'[{j}:{i}] = "{text[lasts[j]:j]} & {substring}" = ({probs[j]:.8f} & {freq:.8f}) = {compounded_prob:.8f}')

        prob_k, k = max(new_probs)  # max of a touple is the max across the first elements, which is the max of the compounded probabilities
        probs.append(prob_k)
        lasts.append(k)

        if debug:
            print(f'i = {i}, prob_k = {prob_k:.8f}, k = {k}, ({text[k:i]})\n')


    # when text is a word that doesn't exist, the algorithm breaks it into individual letters.
    # in that case, return the original word instead
    if len(set(lasts)) == len(text):
        return text

    words = []
    k = len(text)
    while 0 < k:
        word = text[lasts[k]:k]
        words.append(word)
        k = lasts[k]
    words.reverse()
    return ' '.join(words)

def split_message(message):
    new_message = ' '.join(viterbi_segment(wordmash, debug=False) for wordmash in message.split())
    return new_message

In [16]:
df['Title'] = df['Title'].apply(split_message)

### header

In [17]:
def raw(text):
    if 'introduction' in text:
        new = text.split('introduction')
        return new[1] 
    else:
        return ''

def header(text):
    if 'introduction' in text:
        new = text.split('introduction')
        return new[0]   
    else:
        return ''

In [18]:
import re
def abstract(text):

    new = text.split('abstract')
    return new[1]

In [19]:
df['header'] = df['raw_text'].apply(header)

In [20]:
df['raw_text'] = df['raw_text'].apply(raw)

In [21]:
df['abstract'] = df['header'].apply(abstract)

### Punctuation removal

In [22]:
import string
import re 
# import nltk
# from nltk.corpus import stopwords



def pc(data_text):
    data_text = data_text.split('\"' and "," and ":" and '\”' and '\“')
    text = ""
    for i in data_text:
        text = text + i
    print(len(text))

    text = ""

    for i in data_text:
        text = text + i
    
    return text

df['pc_removal'] = df['raw_text'].apply(pc)


36210


### mail

In [23]:
# mail
mail_pattern = re.compile(r'\b\S+@\S+\.\S+\.?\S*\b')
def remove_mail(text):
    return mail_pattern.sub('', text)
df['wmail'] = df['pc_removal'].apply(remove_mail)
# df['without_mail'] = df['without_refs'].apply(remove_mail)

### url

In [24]:
# #url
url_pattern = r'https?://(?:www\.)?[^ ]+|www[^ ]+'
def add_urls(text):
    httplinks = re.findall(url_pattern,text)
    return httplinks

url_pattern = re.compile(url_pattern)
def remove_urls(text):
    return url_pattern.sub('', text)

df['urls'] = df['wmail'].apply(add_urls)
df['wurl'] = df['wmail'].apply(remove_urls)

In [25]:
df['urls'][0]

[]

### 

### split refrences

In [26]:
def ref(text):
    if('references' in text):
        terms = text.split('references')
        return terms[1]
    else:
        return ''
def wref(text):
    if('references' in text):
        terms = text.split('references')
        return terms[0]
    else:
        return text
df['refs'] = df['wurl'].apply(ref)
df['wrefs'] = df['wurl'].apply(wref)

In [27]:
print(df['refs'][0])



[1] kaveh a, farahmand azar b, talatahari s. ant colony optimization for design
of space trusses. international journal of space structures 2008;23(3):167–81.
[2] kaveh a, talatahari s. particle swarm optimizer, ant colony strategy and
harmony search scheme hybridized for optimization of truss structures.
computers and structures 2009;87(5–6):267–83.
[3] he s, wu qh, wen jy, saunders jr, paton rc. a particle swarm optimizer with
passive congregation. biosystem 2004;78:135–47.
[4] kaveh a, talatahari s. a discrete particle swarm ant colony optimization for
design of steel frames. asian journal of civil engineering 2008;9(6):563–75.
[5] kaveh a, talatahari s. a hybrid particle swarm and ant colony optimization
for design of truss structures. asian journal of civil engineering 2008;9(4):
329–48.
[6] coello cac. theoretical and numerical constraint-handling techniques used
with evolutionary algorithms: a survey of the state of the art. computer
methods in applied mechanics and engineerin

### numbers

In [28]:
import re

number_pattern = re.compile(r'\b(?<!\[)(-?\d+(?:\.\d+)?)\b(?![^\[]*\])')

def remove_number(text):
    return number_pattern.sub('', text)

# Assuming df['without_url'] is your DataFrame column containing text
# df['wnumbers'] = df['wrefs'].apply(remove_number)
df['wnumbers'] = df['wrefs']

### sentence segmentation

In [29]:
def sen_seg(data_text):
    sentences = []

    sentence = ''

    # Iterate through the text
    for i in range(len(data_text)):
        char = data_text[i]
        # Append character to the current sentence
        sentence += char
        if char == '.':
            # Check if the character before and after '.' are both numbers
            if i > 0 and i < len(data_text) - 1 and data_text[i-1].isdigit() and data_text[i+1].isdigit():
                continue  # Skip adding '.' to the sentence
            else:
                sentences.append(sentence.strip())
                sentence = ''

    return sentences

# Assuming df['wnumbers'] contains the text data
df['sentences'] = df['wnumbers'].apply(sen_seg)


# Assuming df['wnumbers'] contains the text data
# sen_seg('fjdkfj.fdsfds.s 3.4 dfds.dsa')


In [30]:
print(len(df['sentences'][0]))

312


In [31]:
def checker(sentences):
    new = []
    for s in sentences:
        if(len(s) > 30):
            new.append(s)

    return new

df['sentences'] = df['sentences'].apply(checker)

# selecting sentences with more characters than 30

### merging sentences

In [46]:
def merge_sen(sentences):
    separator = " "
    clean = separator.join(sentences)
    return clean
df['clean'] = df['sentences'].apply(merge_sen)

In [55]:
df.to_csv('preProcessed.csv')